In [1]:
import pandas as pd
import geopandas as gpd
import os
from datetime import datetime, date, timedelta
import numpy as np

In [13]:
path = '/home/rishav/Programs/move_od copy/generated_OD/Williamson_TN_2021-02-01_2021-02-01'

In [14]:
# lodes = pd.read_csv(f"{path}/lodes_combs/lodes_{day}.csv")
lodes = pd.read_csv(f"{path}/lodes_combs/lodes_2021-02-01.csv")
lodes = lodes.rename(
    columns={
        "h_geocode": "home_cbg",
        "w_geocode": "work_cbg",
        "total_jobs": "visits",
        "time_0": "go_time",
        "time_0_str": "go_time_str",
        "time_0_secs": "go_time_secs",
        "time_1": "return_time",
        "time_1_str": "return_time_str",
        "time_1_secs": "return_time_secs",
    }
)

lodes["home_cbg"] = lodes["home_cbg"].astype(str)
# lodes = lodes.merge(chatta[["GEOID"]], left_on="home_cbg", right_on="GEOID")

lodes_go = lodes[
    [
        "home_cbg",
        "work_cbg",
        "home_geom",
        "home_loc_lat",
        "home_loc_lon",
        "work_geom",
        "work_loc_lat",
        "work_loc_lon",
        "visits",
        "go_time",
        "go_time_str",
        "go_time_secs",
    ]
]

lodes_go = lodes_go.rename(
    columns={
        "home_cbg": "pickup_cbg",
        "home_geom": "pickup_geom",
        "home_loc_lat": "pickup_lat",
        "home_loc_lon": "pickup_lon",
        "work_cbg": "dropoff_cbg",
        "work_geom": "dropoff_geom",
        "work_loc_lat": "dropoff_lat",
        "work_loc_lon": "dropoff_lon",
        "go_time": "pickup_time",
        "go_time_str": "pickup_time_str",
        "go_time_secs": "pickup_time_secs",
    }
)

lodes_return = lodes[
    [
        "home_cbg",
        "work_cbg",
        "home_geom",
        "home_loc_lat",
        "home_loc_lon",
        "work_geom",
        "work_loc_lat",
        "work_loc_lon",
        "visits",
        "return_time",
        "return_time_str",
        "return_time_secs",
    ]
]

lodes_return = lodes_return.rename(
    columns={
        "work_cbg": "pickup_cbg",
        "work_geom": "pickup_geom",
        "work_loc_lat": "pickup_lat",
        "work_loc_lon": "pickup_lon",
        "home_cbg": "dropoff_cbg",
        "home_geom": "dropoff_geom",
        "home_loc_lat": "dropoff_lat",
        "home_loc_lon": "dropoff_lon",
        "return_time": "pickup_time",
        "return_time_str": "pickup_time_str",
        "return_time_secs": "pickup_time_secs",
    }
)

lodes_return = lodes_return[lodes_go.columns]

# ADDED TO INDICATE GO AND RETURN TRIPS
lodes_go["direction"] = 0
lodes_return["direction"] = 1

lodes_flattened = pd.concat([lodes_go, lodes_return]).sort_index()

In [15]:
sg = pd.read_csv(f"{path}/safegraph_combs/sg_2021-02-01.csv")
sg = sg.rename(columns={"poi_cbg": "work_cbg"})

sg["home_cbg"] = sg["home_cbg"].astype(str)
# sg = sg.merge(chatta[["GEOID"]], left_on="home_cbg", right_on="GEOID")

sg_go = sg[
    [
        "home_cbg",
        "work_cbg",
        "home_geom",
        "home_loc_lat",
        "home_loc_lon",
        "work_geom",
        "work_loc_lat",
        "work_loc_lon",
        "visits",
        "go_time",
        "go_time_str",
        "go_time_secs",
    ]
]

sg_go = sg_go.rename(
    columns={
        "home_cbg": "pickup_cbg",
        "home_geom": "pickup_geom",
        "home_loc_lat": "pickup_lat",
        "home_loc_lon": "pickup_lon",
        "work_cbg": "dropoff_cbg",
        "work_geom": "dropoff_geom",
        "work_loc_lat": "dropoff_lat",
        "work_loc_lon": "dropoff_lon",
        "go_time": "pickup_time",
        "go_time_str": "pickup_time_str",
        "go_time_secs": "pickup_time_secs",
    }
)

sg_return = sg[
    [
        "home_cbg",
        "work_cbg",
        "home_geom",
        "home_loc_lat",
        "home_loc_lon",
        "work_geom",
        "work_loc_lat",
        "work_loc_lon",
        "visits",
        "return_time",
        "return_time_str",
        "return_time_secs",
    ]
]

sg_return = sg_return.rename(
    columns={
        "work_cbg": "pickup_cbg",
        "work_geom": "pickup_geom",
        "work_loc_lat": "pickup_lat",
        "work_loc_lon": "pickup_lon",
        "home_cbg": "dropoff_cbg",
        "home_geom": "dropoff_geom",
        "home_loc_lat": "dropoff_lat",
        "home_loc_lon": "dropoff_lon",
        "return_time": "pickup_time",
        "return_time_str": "pickup_time_str",
        "return_time_secs": "pickup_time_secs",
    }
)

sg_return = sg_return[sg_go.columns]

# ADDED TO INDICATE GO AND RETURN TRIPS
sg_go["direction"] = 0
sg_return["direction"] = 1

sg_flattened = pd.concat([sg_go, sg_return]).sort_index()

sg_flattened = sg_flattened[sg_flattened['pickup_time_secs']<=86400]

union = pd.concat([lodes_flattened, sg_flattened]).sort_index()
union["pickup_time"] = pd.to_datetime(union["pickup_time"])
union["pickup_hour"] = union["pickup_time"].apply(lambda x: x.hour)
union.to_csv(f"{path}/union_all_trips.csv", index=False)

/tmp/ipykernel_76913/3857073194.py:84: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  union["pickup_time"] = pd.to_datetime(union["pickup_time"])


In [18]:
test = pd.read_csv('/home/rishav/Programs/move_od copy/generated_OD/Rutherford_TN_2021-02-01_2021-02-01/union_all_trips.csv')

In [20]:
test.sort_values('pickup_time_secs').shape

(48173, 15)

In [17]:
test[test['pickup_time_secs'] <=86400].to_csv('/home/rishav/Programs/move_od copy/generated_OD/Rutherford_TN_2021-02-01_2021-02-01/union_all_trips.csv')

In [11]:
bg = gpd.read_file('./data/tl_2022_47_bg.zip')

In [6]:
chatta = gpd.read_file('./data/chattanooga_city_lim.geojson')
chatta_bg = chatta.sjoin(bg)

In [23]:
test.pickup_cbg = test.pickup_cbg.astype(str)
test.dropoff_cbg = test.dropoff_cbg.astype(str)

In [29]:
chatta_od = chatta_bg[['GEOID']].merge(test, left_on = 'GEOID', right_on='pickup_cbg').drop('GEOID', axis=1).merge(chatta_bg[['GEOID']], left_on='dropoff_cbg', right_on = 'GEOID').drop('GEOID', axis=1)
# chatta_od.to_csv('/home/rishav/Programs/move_od copy/generated_OD/Hamilton_TN_2021-02-01_2021-02-01/chatta_all_trips.csv', index=False)

In [16]:
test = pd.read_csv('/home/rishav/Programs/move_od copy/generated_OD/Williamson_TN_2021-02-01_2021-02-01/union_all_trips.csv')

In [19]:
test.sort_values('pickup_time_secs').tail()

,pickup_cbg,dropoff_cbg,pickup_geom,pickup_lat,pickup_lon,dropoff_geom,dropoff_lat,dropoff_lon,visits,pickup_time,pickup_time_str,pickup_time_secs,direction,pickup_hour
6289,471870504062,471870504042,POINT (-86.86964428708308 35.98702209544114),35.987022,-86.869644,POINT (-86.90652151404835 36.0392540815348),36.039254,-86.906522,35.0,2023-06-01 23:58:40,23:58,86320.626174,1,23
19662,471870509052,471870510023,POINT (-86.82500423383594 35.925415969957605),35.925416,-86.825004,POINT (-86.79499749143686 35.82892048760764),35.828920,-86.794997,357.0,2023-06-01 23:58:59,23:58,86339.782711,1,23
21507,471870503071,471870511002,POINT (-86.83139680040502 35.963088057933454),35.963088,-86.831397,POINT (-86.81220979176062 35.7674453692758),35.767445,-86.812210,215.0,2023-06-01 23:59:36,23:59,86376.503243,1,23
1493,471870511001,471870501032,POINT (-86.74795499999999 35.7050475),35.705047,-86.747955,POINT (-86.68080405392061 35.865210204755925),35.865210,-86.680804,257.0,2023-06-01 23:59:57,23:59,86397.263418,1,23
19215,471870509052,471870510023,POINT (-86.82716110239414 35.92621804835769),35.926218,-86.827161,POINT (-86.79499749143686 35.82892048760764),35.828920,-86.794997,357.0,2023-06-01 23:59:59,23:59,86399.782711,1,23


In [20]:
city = gpd.read_file('./data/Franklin_City_Limits.geojson')[['geometry']]

/home/rishav/.local/lib/python3.10/site-packages/geopandas/geodataframe.py:1938: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: EPSG:4326
Right CRS: EPSG:4269

  return geopandas.sjoin(left_df=self, right_df=df, *args, **kwargs)


In [62]:
from shapely.geometry import Polygon, Point

poly =  city['geometry'].iloc[0]
xx, yy, = poly.exterior.coords.xy

coords = []
for x, y in zip(xx, yy) :
    coords.append(Point(float(x), float(y)))

# geoseries = gpd.GeoSeries(coords[:2])

points =  gpd.GeoDataFrame(geometry=coords)
poly = Polygon([[p.x, p.y] for p in coords])
# poly_gdf = gpd.GeoDataFrame(geometry=poly)


In [70]:
city_bg = bg
city_bg['within'] = city_bg['geometry'].apply(lambda x: x.contains(poly))

In [74]:
# city_bg[city_bg['within'] == True]
city_bg.sort_values('within')

,STATEFP,COUNTYFP,TRACTCE,BLKGRPCE,GEOID,NAMELSAD,MTFCC,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,geometry,within
0,47,173,040102,1,471730401021,Block Group 1,G5030,S,52083268,3244698,+36.3228442,-083.7029007,"POLYGON ((-83.76146 36.30071, -83.76145 36.300...",False
3032,47,035,970502,3,470359705023,Block Group 3,G5030,S,3309597,0,+35.9516908,-085.0402165,"POLYGON ((-85.05141 35.95930, -85.05137 35.959...",False
3033,47,035,970702,3,470359707023,Block Group 3,G5030,S,168658140,1395413,+35.8551293,-085.1729843,"POLYGON ((-85.27291 35.78847, -85.27057 35.791...",False
3034,47,093,004700,2,470930047002,Block Group 2,G5030,S,3546189,0,+35.9914610,-084.0257427,"POLYGON ((-84.04483 35.98833, -84.04482 35.988...",False
3035,47,035,970502,2,470359705022,Block Group 2,G5030,S,2329219,0,+35.9386931,-085.0400015,"POLYGON ((-85.05308 35.93720, -85.05303 35.937...",False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1518,47,061,955300,4,470619553004,Block Group 4,G5030,S,39883759,267600,+35.2797170,-085.7728701,"POLYGON ((-85.83813 35.27782, -85.83729 35.278...",False
1517,47,163,043302,2,471630433022,Block Group 2,G5030,S,15129547,86586,+36.4444812,-082.2433416,"POLYGON ((-82.26409 36.44044, -82.26403 36.440...",False
1516,47,179,060501,2,471790605012,Block Group 2,G5030,S,3765183,0,+36.3042210,-082.4126563,"POLYGON ((-82.42925 36.30258, -82.42925 36.302...",False
1522,47,043,060502,3,470430605023,Block Group 3,G5030,S,1123048,0,+36.0822606,-087.3900317,"POLYGON ((-87.40048 36.08575, -87.40020 36.085...",False


In [76]:
city_bg = bg.sjoin()

InvalidIndexError: POLYGON ((-86.5788521416367 36.7416284454649, -86.5788392428849 36.7413993207298, -86.5797060622704 36.7397018038138, -86.5800021564935 36.7391283790968, -86.5800780002068 36.7386609997125, -86.5802297032133 36.7386615867556, -86.5787171891988 36.7384039416869, -86.5787340028405 36.7388354676676, -86.5777137764628 36.7388917423774, -86.5766260366788 36.7389942615811, -86.5755730061336 36.7390930490046, -86.5756311985941 36.7402842836344, -86.5755360707337 36.7417327380496, -86.5732547834517 36.7418345892766, -86.5729591025596 36.7418425390308, -86.572488449612 36.7442706097249, -86.578434999119 36.743860000353, -86.5784650046491 36.7441491386683, -86.5789800743752 36.744149039526, -86.5789672965872 36.7442738968734, -86.5782710001103 36.7443230001025, -86.5782539996431 36.7450650000524, -86.5775421036061 36.7451587279635, -86.577244892399 36.7451880440359, -86.5769070790181 36.7452176453645, -86.5764744976798 36.7452370008383, -86.5757706186871 36.745284524068, -86.5

In [ ]:
test.pickup_cbg = test.pickup_cbg.astype(str)
test.dropoff_cbg = test.dropoff_cbg.astype(str)

In [ ]:
city_od = city_bg[['GEOID']].merge(test, left_on = 'GEOID', right_on='pickup_cbg').drop('GEOID', axis=1).merge(city_bg[['GEOID']], left_on='dropoff_cbg', right_on = 'GEOID').drop('GEOID', axis=1)
# city_od.to_csv('/home/rishav/Programs/move_od copy/generated_OD/Hamilton_TN_2021-02-01_2021-02-01/city_all_trips.csv', index=False)